COMP-6651 - Algorithm Design Techniques

Project - Clustering Algorithms Analysis



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift as SklearnMeanShift
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import os
import kagglehub
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import (calinski_harabasz_score, silhouette_score,
                             davies_bouldin_score, adjusted_rand_score,
                             normalized_mutual_info_score)

In [ ]:
def preprocess_ai_index(data):
    # Separate numerical and categorical columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns

    scaler = MinMaxScaler()
    num_scaled = scaler.fit_transform(data[numeric_cols]) if len(numeric_cols) > 0 else np.array([])

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cat_encoded = encoder.fit_transform(data[categorical_cols]) if len(categorical_cols) > 0 else np.array([])

    if num_scaled.size and cat_encoded.size:
        return np.hstack((num_scaled, cat_encoded))
    elif num_scaled.size:
        return num_scaled
    else:
        return cat_encoded


In [ ]:
path_iris = kagglehub.dataset_download("himanshunakrani/iris-dataset")
f_path_iris = os.path.join(path_iris, 'iris.csv')
iris_df = pd.read_csv(f_path_iris)
# Extract labels (species)
iris_labels = iris_df['species']
# Remove label from the features
iris_features = iris_df.drop(columns=['species'])
# Convert to numpy
iris_data = iris_features.values



path_ai_index = kagglehub.dataset_download("katerynameleshenko/ai-index")
f_path_ai_index = os.path.join(path_ai_index, 'AI_index_db.csv')
ai_df = pd.read_csv(f_path_ai_index)
ai_df = ai_df.dropna()
# Extract the 'Cluster' column as the label
ai_labels = ai_df['Cluster']
# Remove the 'Cluster' column from the features
ai_df_features = ai_df.drop(columns=['Cluster'])
# Preprocess the remaining features
ai_data = preprocess_ai_index(ai_df_features)

path_earthquakes= kagglehub.dataset_download("shreyasur965/recent-earthquakes")
f_path_earthquakes = os.path.join(path_earthquakes, 'earthquakes.csv')
earthquake_df = pd.read_csv(f_path_earthquakes)
earthquake_df = earthquake_df[['magnitude', 'felt', 'cdi','mmi','tsunami','sig','depth', 'latitude', 'longitude', 'alert']].dropna()
# Extract the alert labels
earthquake_data_alerts = earthquake_df['alert']
alert_encoded = LabelEncoder().fit_transform(earthquake_data_alerts)
# Remove the label from features
earthquake_data_features = earthquake_df.drop(columns=['alert'])
# Scale the numeric features
earthquake_data = StandardScaler().fit_transform(earthquake_data_features)

100%|██████████| 0.98k/0.98k [00:00<00:00, 1.82MB/s]

Extracting files...


100%|██████████| 2.38k/2.38k [00:00<00:00, 3.64MB/s]

Extracting files...


100%|██████████| 214k/214k [00:00<00:00, 43.5MB/s]

Extracting files...


In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances

class MeanShift:
    def __init__(self, bandwidth=0.5, convergence_thresh=1e-3, max_iter=100):
        self.bandwidth = bandwidth  # Renamed from sigma to match conventions
        self.convergence_thresh = convergence_thresh  # Renamed from epsilon
        self.max_iter = max_iter
        self.labels_ = None
        self.cluster_centers_ = None

    def fit(self, X):
        X = np.asarray(X)
        n_samples, _ = X.shape
        modes = np.zeros_like(X)

        # Step 1: Compute modes for each point
        for i in range(n_samples):
            x_old = X[i].copy()
            for _ in range(self.max_iter):
                # Compute weights using Gaussian kernel
                distances = np.sum((X - x_old) ** 2, axis=1)
                weights = np.exp(-distances / (2 * self.bandwidth ** 2))
                weights_sum = np.sum(weights)

                if weights_sum < 1e-10:  # Handle zero division
                    break

                # Update mean-shift vector
                x_new = np.dot(weights, X) / weights_sum

                # Check convergence
                if np.linalg.norm(x_new - x_old) < self.convergence_thresh:
                    break
                x_old = x_new
            modes[i] = x_old

        # Step 2: Merge modes using adaptive epsilon
        self._auto_merge_modes(modes)
        return self

    def _auto_merge_modes(self, modes):
        """Automatically determine merge threshold and cluster modes"""
        # Compute pairwise distances between modes
        mode_distances = pairwise_distances(modes)

        # Use 10th percentile of non-zero distances as epsilon
        non_zero_dists = mode_distances[mode_distances > 0]
        if len(non_zero_dists) == 0:
            self.epsilon = 0.0
        else:
            self.epsilon = np.percentile(non_zero_dists, 20)  # 5th percentile

        # Apply connected components
        self.labels_ = self._connected_components(modes, self.epsilon)

        # Extract unique cluster centers
        unique_labels = np.unique(self.labels_)
        self.cluster_centers_ = np.array([
            modes[self.labels_ == label][0]
            for label in unique_labels
        ])

    def _connected_components(self, modes, epsilon):
        """Optimized union-find with path compression"""
        n = len(modes)
        parent = list(range(n))

        # Precompute all pairs within epsilon
        dist_matrix = pairwise_distances(modes)
        rows, cols = np.where((dist_matrix <= epsilon) & (np.eye(n) == 0))

        # Union-Find operations
        def find(u):
            while parent[u] != u:
                parent[u] = parent[parent[u]]  # Path compression
                u = parent[u]
            return u

        for i, j in zip(rows, cols):
            if i < j:
                root_i = find(i)
                root_j = find(j)
                if root_i != root_j:
                    parent[root_j] = root_i

        # Label clusters
        labels = np.empty(n, dtype=int)
        label_map = {}
        current_label = 0
        for i in range(n):
            root = find(i)
            if root not in label_map:
                label_map[root] = current_label
                current_label += 1
            labels[i] = label_map[root]

        return labels

In [ ]:
def compute_diameter(X, labels):
    """
    Computes the average 'diameter' across all clusters,
    where 'diameter' of a cluster is the maximum distance
    between any two points in that cluster.
    """
    from scipy.spatial.distance import pdist, squareform

    unique_labels = np.unique(labels)
    diameters = []
    for lbl in unique_labels:
        cluster_points = X[labels == lbl]
        if len(cluster_points) > 1:
            # pairwise distances in the cluster
            dist_matrix = squareform(pdist(cluster_points))
            diameters.append(dist_matrix.max())
        else:
            # A single point has diameter 0
            diameters.append(0.0)
    return np.mean(diameters)


def compute_split(X, labels):
    """
    An example 'split' metric: ratio of the size of the largest cluster
    to the size of the smallest cluster. If there's only one cluster, return 1.
    """
    unique_labels, counts = np.unique(labels, return_counts=True)
    if len(counts) < 2:
        return 1.0
    return counts.max() / counts.min()


def evaluate_clustering(X, labels, true_labels=None):
    """
    Compute a set of metrics for the given clustering labels.
    Some metrics require ground truth (true_labels).
    If no true_labels is provided, ARI and MI will be omitted.
    Returns a dict of metric_name -> value.
    """
    metrics_dict = {}

    # Unsupervised metrics
    metrics_dict["Silhouette"] = silhouette_score(X, labels)
    metrics_dict["Davies-Bouldin"] = davies_bouldin_score(X, labels)
    metrics_dict["Calinski-Harabasz"] = calinski_harabasz_score(X, labels)
    metrics_dict["Diameter"] = compute_diameter(X, labels)
    metrics_dict["Split"] = compute_split(X, labels)

    # If we have ground-truth labels, we can compute supervised metrics
    if true_labels is not None:
        metrics_dict["Adjusted Rand Index"] = adjusted_rand_score(true_labels, labels)
        # Using normalized_mutual_info_score as a measure of MI
        metrics_dict["Mutual Information"] = normalized_mutual_info_score(true_labels, labels)

    return metrics_dict


In [ ]:

def run_experiment(name, X, bandwidth=None, max_iter=300, convergence_thresh=1e-3,
                             true_labels=None):
    print(f"----- {name} -----")

    # Custom Mean Shift Implementation
    custom_ms = MeanShift(
        bandwidth=bandwidth,
        max_iter=max_iter
    )
    custom_ms.fit(X)
    custom_labels = custom_ms.labels_
    n_custom_clusters = len(custom_ms.cluster_centers_)
    print(f"Custom MS: Number of clusters = {n_custom_clusters}")

    # Scikit-learn's Implementation
    sklearn_ms = SklearnMeanShift(
        bandwidth=bandwidth,
        max_iter=max_iter,
        bin_seeding=False,
        min_bin_freq=1,
        cluster_all=True
    )
    sklearn_ms.fit(X)
    sklearn_labels = sklearn_ms.labels_
    n_sklearn_clusters = len(sklearn_ms.cluster_centers_)
    print(f"Sklearn MS: Number of clusters = {n_sklearn_clusters}")


    custom_metrics = evaluate_clustering(X, custom_labels, true_labels=true_labels)
    sklearn_metrics = evaluate_clustering(X, sklearn_labels, true_labels=true_labels)

    print("\nCustom MS Metrics:")
    for k, v in custom_metrics.items():
        print(f"  {k}: {v:.4f}")

    print("\nSklearn MS Metrics:")
    for k, v in sklearn_metrics.items():
        print(f"  {k}: {v:.4f}")

    print("-" * 40)
    return custom_labels, sklearn_labels

In [ ]:
from sklearn.cluster import estimate_bandwidth
bandwidth = estimate_bandwidth(iris_data, quantile=0.4)


print("Running Custom and Sklearn MS on Iris Dataset")
custom_labels_iris, sklearn_labels_iris = run_experiment(
    "Iris", iris_data,
    bandwidth=bandwidth,  # Controls cluster granularity
    true_labels=iris_labels
)
bandwidth = estimate_bandwidth(ai_data, quantile=0.4)

print("\nRunning Custom and Sklearn MS on AI Global Index Dataset")
custom_labels_ai, sklearn_labels_ai = run_experiment(
    "AI Global Index", ai_data,
    bandwidth=bandwidth,  # Larger bandwidth for broader clusters
    true_labels=ai_labels
)
bandwidth = estimate_bandwidth(earthquake_data, quantile=0.4)

print("\nRunning Custom and Sklearn MS on Earthquake Dataset")
custom_labels_eq, sklearn_labels_eq = run_experiment(
    "Earthquake", earthquake_data,
    bandwidth=bandwidth,  # Smaller bandwidth for fine-grained clusters
    true_labels=alert_encoded
)



Running Custom and Sklearn MS on Iris Dataset
----- Iris -----
Custom MS: Number of clusters = 2
Sklearn MS: Number of clusters = 2

Custom MS Metrics:
  Silhouette: 0.5025
  Davies-Bouldin: 0.6699
  Calinski-Harabasz: 266.7661
  Diameter: 3.6179
  Split: 1.1429
  Adjusted Rand Index: 0.4240
  Mutual Information: 0.4822

Sklearn MS Metrics:
  Silhouette: 0.6855
  Davies-Bouldin: 0.3893
  Calinski-Harabasz: 508.8822
  Diameter: 3.6903
  Split: 1.9412
  Adjusted Rand Index: 0.5584
  Mutual Information: 0.6994
----------------------------------------

Running Custom and Sklearn MS on AI Global Index Dataset
----- AI Global Index -----
Custom MS: Number of clusters = 7
Sklearn MS: Number of clusters = 2

Custom MS Metrics:
  Silhouette: 0.1175
  Davies-Bouldin: 1.5615
  Calinski-Harabasz: 5.9750
  Diameter: 1.9084
  Split: 20.0000
  Adjusted Rand Index: 0.0179
  Mutual Information: 0.2412

Sklearn MS Metrics:
  Silhouette: 0.1629
  Davies-Bouldin: 0.7229
  Calinski-Harabasz: 1.7988
  Diame

In [ ]:
def select_top_k_features_for_alert(df, alert_col='alert', k=5):
    """
    Filters features based on Mutual Information (MI) with the alert label.
    Steps:
      1) Drop rows with any missing values (to avoid errors).
      2) Label-encode the alert column if it is categorical (e.g., text).
      3) For each feature, compute MI against the alert.
      4) Sort features by MI score, descending.
      5) Return the top k feature names.

    """
    data = df.dropna()

    # Separate the target (alert) from the features
    y = data[alert_col]
    X_df = data.drop(columns=[alert_col])

    # encode alert:
    y_enc = LabelEncoder().fit_transform(y)

    # mutual_info_classif calculates MI between each column and y_enc
    mi_scores = mutual_info_classif(X_df.values, y_enc, discrete_features=False)

    # Pair up (feature_name, mi_score)
    feats_and_scores = list(zip(X_df.columns, mi_scores))

    # Sort by MI descending
    feats_and_scores.sort(key=lambda x: x[1], reverse=True)

    # Pick top k
    top_features = [f[0] for f in feats_and_scores[:k]]

    return top_features

## 4.2 - Select the top 5 features for predicting alert

In [ ]:
top5_features = select_top_k_features_for_alert(
    earthquake_df,  # the original DataFrame with 'alert' and other columns
    alert_col='alert',
    k=5
)
print("Top 5 features most predictive of 'alert':", top5_features)

Top 5 features most predictive of 'alert': ['sig', 'mmi', 'longitude', 'latitude', 'magnitude']


## 4.3 - Re-run the clustering using only top-5 features

In [ ]:
# Subset the DataFrame to those top-5 columns
from sklearn.cluster import estimate_bandwidth

earthquake_df_top5 = earthquake_df[top5_features]
bandwidth = estimate_bandwidth(earthquake_data, quantile=0.4)

# 2) Scale the new subset
eq_top5_data = StandardScaler().fit_transform(earthquake_df_top5.values)

print("Running Base/Sklearn Affinity on Earthquake Dataset with Top-5 Features")
_ = run_experiment(
    "Earthquake (Top-5 Features)",
    eq_top5_data,
    bandwidth=bandwidth,  # Smaller bandwidth for fine-grained clusters
    true_labels=alert_encoded
)


Running Base/Sklearn Affinity on Earthquake Dataset with Top-5 Features
----- Earthquake (Top-5 Features) -----
Custom MS: Number of clusters = 17
Sklearn MS: Number of clusters = 2

Custom MS Metrics:
  Silhouette: -0.3665
  Davies-Bouldin: 1.5267
  Calinski-Harabasz: 3.2254
  Diameter: 0.8234
  Split: 664.0000
  Adjusted Rand Index: -0.0865
  Mutual Information: 0.0217

Sklearn MS Metrics:
  Silhouette: 0.6368
  Davies-Bouldin: 0.5429
  Calinski-Harabasz: 139.2829
  Diameter: 6.2431
  Split: 62.6667
  Adjusted Rand Index: 0.3440
  Mutual Information: 0.3464
----------------------------------------
